## STEP 1 - Importing Required Libraries

In [ ]:
import cv2
import numpy as np
from PIL import Image
from numpy import asarray 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from skimage import filters, feature, measure, color
from skimage.segmentation import watershed
from scipy import ndimage
from google.colab.patches import cv2_imshow

## STEP 2- Image Pre-processing

In [ ]:
img=cv2.imread("/content/rice_img.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

cv2_imshow(img)
cv2_imshow(gray)

In [ ]:
ret,thresh=cv2.threshold(gray,120,255,cv2.THRESH_BINARY)
cv2_imshow(thresh)

##Noise Removal

In [ ]:
kernal=np.ones((3),np.uint8)
clear_img=cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernal,iterations=8)

cv2_imshow(clear_img)

## STEP 3-Counting the Labels

In [ ]:
#Copying the preprocessed images as label_img
label_img=clear_img.copy()

label_count=0

#shape function for getting height and width
rows,cols=label_img.shape

#looping through the pixel of image using rows and column of image
for j in range(rows):
  for i in range(cols):
    pixel=label_img[j,i]

    #Here there is single channel with 2pixel intensities,Either 0 or 255
    #Counting the total number of pixel intensity 255
    if 255==pixel:
      label_count+=1
      ##Applying floodFill method of opencv which will help in filling the backgroud that will ultimately helps in couting the contoured grain ea
      cv2.floodFill(label_img,None,(i,j),label_count)
      

In [ ]:
print("Number of foreground objects",label_count)
cv2_imshow(label_img)

## Counting rice grains using Contours method

In [ ]:
#Applying Countours method to get the count of rice grains
contours,h=cv2.findContours(clear_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

output_contour=cv2.cvtColor(clear_img,cv2.COLOR_GRAY2BGR)
cv2.drawContours(output_contour,contours,-1,(0,0,255),2)
print("Number of detected contours",len(contours))

In [ ]:
cv2_imshow(output_contour)

## STEP 3-Applying Watershed Algorithm

In [ ]:
#To visualize the segmentation conveniently, There needed a colour-code the labelled regions using the color, thus I did it
#Applying  distance_transform_edt to computes the distance from non-zero (i.e. non-background) points to the nearest zero (i.e. background) point.

dist_trans=ndimage.distance_transform_edt(clear_img)

#Applying peak_local_max function for getting coordinates of local peaks (maxima) in an image.
local_max=feature.peak_local_max(dist_trans,min_distance=23)

local_max_mask=np.zeros(dist_trans.shape,dtype=bool)
local_max_mask[tuple(local_max.T)]=True

#Aplying Watershed algorithm
labels=watershed(-dist_trans,measure.label(local_max_mask),mask=clear_img)



## STEP 5-Counting Total Grains and Broken grains using grains area

In [ ]:
#label2rgb function, specifying the background label with argument bg_label=0.
plt.figure(figsize=(30,10))
plt.imshow(color.label2rgb(labels,bg_label=0))
print("Number of Rice grains are :%d" % labels.max())

## Count the Total number of Rice grains (Done) 

In [ ]:
#Creating A list name count for counting the total Broken rice grains
count=[]

#Iterating through contour and filtering out the rice grains with area less than 800 and then appending to the count variable.
for x in contours:

  #Using contourArea method to get the area of rice (Contour)
  area=cv2.contourArea(x)
  if area<800:
    count.append(x)

print("Total Number of Broken rice present in the Image is:",len(count))